In [ ]:
%git clone https://github.com/ultralytics/yolov5.git

In [ ]:
%pip install -r yolov5/requirements.txt

In [ ]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [ ]:
%mkdir dataset
%cd dataset
%wget -O RoadSignDetectionDataset.zip https://arcraftimages.s3-accelerate.amazonaws.com/Datasets/CarsAndTrafficSigns/CarsAndTrafficSignsPascalVOC.zip?region=us-east-2
%unzip RoadSignDetectionDataset.zip
%rm -r __MACOSX RoadSignDetectionDataset.zip
%mv annotations labels

In [ ]:
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []
    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            info_dict['image_size'] = tuple(image_size)
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    return info_dict
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"traffic sign": 0,
                            "traffic light": 1,
                            "person": 2,
                            "car": 3,
                            "truck": 4,
                            "bike": 5,
                            "rider": 6,
                            "bus": 7,
                            "motor": 8,
                            "train": 9}
# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []
    MIN = 15 # Minimum width and height of the bounding box
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
            # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        if b_width > MIN and b_height > MIN and class_id in [0, 1]:
            # Normalise the co-ordinates by the dimensions of the image
            image_w, image_h, image_c = info_dict["image_size"]  
            b_center_x /= image_w 
            b_center_y /= image_h 
            b_width    /= image_w 
            b_height   /= image_h 
            #Write the bbox details to the file 
            print_buffer.append("{} {:.5f} {:.5f} {:.5f} {:.5f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
            print("{} {:.5f} {:.5f} {:.5f} {:.5f}".format(class_id, b_center_x, b_center_y, b_width, b_height))    
    # Name of the file which we have to save 
    save_file_name = os.path.join("labels", info_dict["filename"].replace("png", "txt"))
    # Save the label to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))
# Get the labels
labels = [os.path.join('labels', x) for x in os.listdir('labels') if x[-3:] == "xml"]
labels.sort()
# Convert and save the labels
for ann in tqdm(labels):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)
labels = [os.path.join('labels', x) for x in os.listdir('labels') if x[-3:] == "txt"]
class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

In [ ]:
%mkdir images/train images/val images/test labels/train labels/val labels/test

In [ ]:
# Read images and labels
images = [os.path.join('images', x) for x in os.listdir('images') if x[-3:] == "png"].sort()
labels = [os.path.join('labels', x) for x in os.listdir('labels') if x[-3:] == "txt"].sort()
# Split the dataset into train-valid-test splits 
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size = 0.2, random_state = 1)
val_images, test_images, val_labels, test_labels = train_test_split(val_images, val_labels, test_size = 0.5, random_state = 1)
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False
    print(destination_folder, "done")
# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_labels, 'labels/train/')
move_files_to_folder(val_labels, 'labels/val/')
move_files_to_folder(test_labels, 'labels/test/')

In [ ]:
# Get the labels
labels = [os.path.join('labels', x) for x in os.listdir('labels') if x[-3:] == "xml"]
labels.sort()

# Convert and save the labels
for ann in tqdm(labels):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)
labels = [os.path.join('labels', x) for x in os.listdir('labels') if x[-3:] == "txt"]

In [ ]:
random.seed(0)

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

def plot_bounding_box(image, annotation_list):
    labels = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_labels = np.copy(labels)
    transformed_labels[:,[1,3]] = labels[:,[1,3]] * w
    transformed_labels[:,[2,4]] = labels[:,[2,4]] * h 
    
    transformed_labels[:,1] = transformed_labels[:,1] - (transformed_labels[:,3] / 2)
    transformed_labels[:,2] = transformed_labels[:,2] - (transformed_labels[:,4] / 2)
    transformed_labels[:,3] = transformed_labels[:,1] + transformed_labels[:,3]
    transformed_labels[:,4] = transformed_labels[:,2] + transformed_labels[:,4]
    
    for ann in transformed_labels:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)))
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))])
    
    plt.imshow(np.array(image))
    plt.show()

# Get any random annotation file 
annotation_file = random.choice(labels)
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]

#Get the corresponding image file
image_file = annotation_file.replace("labels", "images").replace("txt", "png")
assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)

#Plot the Bounding Box
plot_bounding_box(image, annotation_list)

In [ ]:
%cd ..
%cp hyp.traffic.yaml yolov5/data/hyps/hyp.traffic.yaml
%cp traffic.yaml yolov5/data/traffic.yaml
%cd yolov5

In [ ]:
%python3 train.py --img 640 --cfg yolov5s.yaml --data traffic.yaml --weights yolov5s.pt --hyp hyp.traffic.yaml --batch-size -1 --epochs 300 --name traffic --patience 50 --save-period 10 --cache

In [ ]:
%python3 detect.py --source ../datasets/traffic/images/test/ --weights runs/train/traffic/weights/best.pt --conf 0.25 --name traffic

In [ ]:
detections_dir = "runs/detect/traffic/"
detection_images = [os.path.join(detections_dir, x) for x in os.listdir(detections_dir)]

random_detection_image = Image.open(random.choice(detection_images))
plt.imshow(np.array(random_detection_image))

In [ ]:
%python3 val.py --weights runs/train/traffic/weights/best.pt --data traffic.yaml --task test --name traffic

In [ ]:
%python export.py --weights runs/train/yolo_road_det3/weights/best.pt --include tflite